In [1]:
# Import Data

In [ ]:
'''
Raw data imported.
Censored due to privacy concerns.
'''

In [ ]:
from Data_Preprocess_EEGNet import Data_Preprocess_EEGNet
DPE = Data_Preprocess_EEGNet(data,info)
PP_data, label_ndarrays, label_ndarrays_CInonCI = DPE.epoch_based_organizing(z_score=True)

def save_results_EEGNet(Y, evaluation, folder_path, csv_name:str):
    import pandas as pdpandas
    import os
    folder_path = folder_path
    # Create the folder if it does not exist
    if not os.path.exists(folder_path):    os.makedirs(folder_path)
        
    evaluation_DF = pd.DataFrame(evaluation, index=[0])
    evaluation_DF.to_csv(os.path.join(folder_path, f'evaluation_{csv_name}.csv'), index=False)

    Y_DF = pd.DataFrame(Y)
    Y_DF.to_csv(os.path.join(folder_path, f'Y_{csv_name}.csv'), index=False)

# EEGNet
- **First apply train-validation-test**
- **switch the sequence of train and valid dataset**
    - To see whether the EEGNet model remenber the information of train dataset which is fit in the previous model training   


## train-validation-test

In [2]:
from EEGNet_function import EEGNet
from tensorflow.keras.optimizers import Adam

model = EEGNet(input_shape=(1, 30, 375),
               num_class=2,
               loss='sparse_categorical_crossentropy',
               optimizer=Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08),
               epochs=100,
               batch_size=100,
               es_patience=50,
               monitor='val_accuracy',
               log_path = 'logs/fold1',
               shuffle = True)

model.fit(PP_data['train'], label_ndarrays_CInonCI['train'], PP_data['valid'], label_ndarrays_CInonCI['valid'])

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
Y,evaluation = model.predict( PP_data['valid'], label_ndarrays_CInonCI['valid'])

print(evaluation)

folder_path = f'./Results/EEGNet/TVT'

save_results_EEGNet(Y, evaluation, folder_path, 'valid')

In [ ]:
Y,evaluation = model.predict(PP_data['test'], label_ndarrays_CInonCI['test'])

print(evaluation)

folder_path = f'./Results/EEGNet/TVT'

save_results_EEGNet(Y, evaluation, folder_path, 'test')

## Train <-> valid

In [ ]:
# from EEGNet_function import EEGNet
model = EEGNet(input_shape=(1, 30, 375),
               num_class=2,
               loss='sparse_categorical_crossentropy',
               optimizer=Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08),
               epochs=100,
               batch_size=100,
               es_patience=50,
               monitor='val_accuracy',
               log_path= 'logs/Fold2',
               shuffle = True)

model.fit(PP_data['valid'], label_ndarrays_CInonCI['valid'], PP_data['train'], label_ndarrays_CInonCI['train'])

In [ ]:
Y,evaluation = model.predict( PP_data['valid'], label_ndarrays_CInonCI['valid'])

folder_path = f'./Results/EEGNet/TVT_TVswitch'

save_results_EEGNet(Y, evaluation, folder_path, 'valid')

Y,evaluation = model.predict( PP_data['train'], label_ndarrays_CInonCI['train'])

folder_path = f'./Results/EEGNet/TVT_TVswitch'

save_results_EEGNet(Y, evaluation, folder_path, 'train')

In [ ]:
model.predict( PP_data['test'], label_ndarrays_CInonCI['test'])

folder_path = f'./Results/EEGNet/TVT_TVswitch'

save_results_EEGNet(Y, evaluation, folder_path, 'test')

## 5-Fold test

### import data

In [ ]:
from hnc import function_hnc

path_data = '../data/Dementia_paper_dataset_rawData.hdf5'
data = {}
data['train'] = function_hnc.hnc_load_data(path_data, 'train/data')
data['valid'] = function_hnc.hnc_load_data(path_data, 'valid/data')
data['test'] = function_hnc.hnc_load_data(path_data, 'test/data')

path_info = '../data/Dementia_paper_dataset_info.pkl'
info = {}
info['train'] = function_hnc.hnc_load_data(path_info, 'train')
info['valid'] = function_hnc.hnc_load_data(path_info, 'valid')
info['test'] = function_hnc.hnc_load_data(path_info, 'test')

In [ ]:
import numpy as np
data['train_valid'] = np.concatenate([data['train'], data['valid']]) 
data['train_valid'].shape
import pandas as pd
info['train_valid'] = pd.concat([info['train'], info['valid']]) 
info['train_valid'].shape
print(info.keys())
print(data.keys())
keys_to_remove = ['train', 'valid']
for key in keys_to_remove:
    info.pop(key)
    data.pop(key)
print(info.keys())
print(data.keys())

### functions

In [ ]:
def subject_based_accuracy(Y, nEpoch_perSub = 40):
    idx = 0
    subject_correct = []
    subject_true_label = []
    subject_pred_label = []
    nEpoch = Y['y_true'].shape[0]
    while idx+nEpoch_perSub <= nEpoch:
        subject_true_label.append(np.average(Y['y_true'][idx:idx+nEpoch_perSub])) # access the true label
        if np.average(Y['y_true'][idx:idx+nEpoch_perSub] == Y['y_pred'][idx:idx+nEpoch_perSub]) > 0.5:
            subject_correct.append(1)
            subject_pred_label.append(np.average(Y['y_true'][idx:idx+nEpoch_perSub])) # access the predict label
        else:
            subject_correct.append(0)
            subject_pred_label.append(np.abs(1-np.average(Y['y_true'][idx:idx+nEpoch_perSub]))) # access the predict label
        idx += nEpoch_perSub

    
    subject_correct = np.array(subject_correct)
    subject_true_label = np.array(subject_true_label)
    subject_pred_label = np.array(subject_pred_label)
    
    accuracy = np.average(subject_correct)
    return subject_correct, accuracy, subject_true_label, subject_pred_label

def access_label(info):
    label_ndarrays = {}
    label_ndarrays_CInonCI = {}
    for key, value in info.items():
        if key != 'Info':
            label_ndarrays[key] = value['Label'].values
            label_ndarrays_CInonCI[key] = value['Label'].values
        else:
            label_ndarrays[key] = value
            label_ndarrays_CInonCI[key] = value

    # Replace 2 with 1 in label_ndarrays
    label_ndarrays_CInonCI = {key: np.where(value == 2, 1, value) for key, value in label_ndarrays_CInonCI.items()}

    return label_ndarrays_CInonCI

def save_results_EEGNet(Y, evaluation, folder_path, csv_name:str):
    import pandas as pdpandas
    import os
    folder_path = folder_path
    # Create the folder if it does not exist
    if not os.path.exists(folder_path):    os.makedirs(folder_path)
        
    evaluation_DF = pd.DataFrame(evaluation, index=[0])
    evaluation_DF.to_csv(os.path.join(folder_path, f'evaluation_{csv_name}.csv'), index=False)

    Y_DF = pd.DataFrame(Y)
    Y_DF.to_csv(os.path.join(folder_path, f'Y_{csv_name}.csv'), index=False)

### Grid Search Hyperparams. or preprocessing

In [3]:
from sklearn.model_selection import ParameterGrid

# scaling = ['z_score', 'minmax', 'none']
# conduct_Czrrf = [True, False]
# at_each_time_point = [True, False]
scaling = ['minmax']
conduct_Czrrf = [True, False]
at_each_time_point = [True]
across_channel = [True]

if scaling == 'z_score':
    z_score = True
    minmax = False
elif scaling == 'minmax':
    z_score = False
    minmax = True 
else:
    z_score = False
    minmax = False 

# Create multiple parameter set based on all posible combination of the parameter grids
param_grid = [
                 {'scaling': ['z_score', 'minmax', 'none'], 
                  'conduct_Czrrf': [True, False], 
                  'at_each_time_point': [True],
                  'across_channel': [True],
                  'dropout_rate': [0.5],
                  'lr': [0.001, 0.01]},
                 {'scaling':  ['z_score', 'minmax'], 
                  'conduct_Czrrf': [True, False],
                  'at_each_time_point': [False],
                  'across_channel': [True, False],
                  'dropout_rate': [0.5],
                  'lr': [0.001, 0.01]},
                ]
param_grid_list = list(ParameterGrid(param_grid))
param_grid_list

[{'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.001,
  'scaling': 'z_score'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.001,
  'scaling': 'minmax'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.001,
  'scaling': 'none'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.01,
  'scaling': 'z_score'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.01,
  'scaling': 'minmax'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': True,
  'dropout_rate': 0.5,
  'lr': 0.01,
  'scaling': 'none'},
 {'across_channel': True,
  'at_each_time_point': True,
  'conduct_Czrrf': False,
  'dropout_rate': 0.5,
  'lr': 0.001,
  'scaling': 'z_score'},
 {

### Main

In [ ]:
from sklearn.model_selection import ParameterGrid

# scaling = ['z_score', 'minmax', 'none']
# conduct_Czrrf = [True, False]
# at_each_time_point = [True, False]
scaling = ['minmax']
conduct_Czrrf = [True, False]
at_each_time_point = [True]
across_channel = [True]

if scaling == 'z_score':
    z_score = True
    minmax = False
elif scaling == 'minmax':
    z_score = False
    minmax = True 
else:
    z_score = False
    minmax = False 

# Create multiple parameter set based on all posible combination of the parameter grids
param_grid = [
                 {'scaling': ['z_score', 'minmax', 'none'], 
                  'conduct_Czrrf': [True, False], 
                  'at_each_time_point': [True],
                  'across_channel': [True],
                  'dropout_rate': [0.5],
                  'lr': [0.001, 0.01]},
                 {'scaling':  ['z_score', 'minmax'], 
                  'conduct_Czrrf': [True, False],
                  'at_each_time_point': [False],
                  'across_channel': [True, False],
                  'dropout_rate': [0.5],
                  'lr': [0.001, 0.01]},
                ]
param_grid_list = list(ParameterGrid(param_grid))
param_grid_list

### Independant test: 
model were trained by all training data (data['train_valid'])

In [4]:
from sklearn.model_selection import StratifiedKFold
from EEGNet_function import EEGNet
from tensorflow.keras.optimizers import Adam
from Data_Preprocess_EEGNet import Data_Preprocess_EEGNet
import os
import pickle

# apply grid search for optimal hyperparameters
# grid = {'dropout_rate': [0.5],
#         'lr': [0.001, 0.0001]}


for idx_grid_search, params in enumerate(param_grid_list):
    print('params:', params)
    dropout_rate = params['dropout_rate']
    conduct_Czrrf = params['conduct_Czrrf']
    lr = params['lr']
    at_each_time_point = params['at_each_time_point']
    if at_each_time_point == True:
        across_channel = True
    else:
        across_channel = params['across_channel']
        
    scaling = params['scaling']
    if scaling == 'z_score':
        z_score = True
        minmax = False
    elif scaling == 'minmax':
        z_score = False
        minmax = True 
    else:
        z_score = False
        minmax = False    
    
        
    # preprocess
    Freq = target_Freq
    if Freq == 500:
        DPE = Data_Preprocess_EEGNet(data, info, conduct_decimate=False, conduct_Czrrf=conduct_Czrrf, across_channel=across_channel,
                                    conduct_filter=conduct_filter, band=band, at_each_time_point=at_each_time_point,
                                    sec_Window=sec_Window, sec_Overlap=sec_Overlap)
    else:
        DPE = Data_Preprocess_EEGNet(data, info, target_Freq=Freq, conduct_Czrrf=conduct_Czrrf, across_channel=across_channel,
                                    conduct_filter=conduct_filter, band=band, at_each_time_point=at_each_time_point,
                                    sec_Window=sec_Window, sec_Overlap=sec_Overlap)
    
    PP_data, label_ndarrays, label_ndarrays_CInonCI = DPE.epoch_based_organizing(z_score=z_score, minmax=minmax)
    segment_shape = PP_data['train_valid'][0].shape
    
    Y_perFold = {'train_valid':list(),'test':list()}
    evaluation_perFold = {'train_valid':list(),'test':list()}
    
    # Define the model architecture
    k_length = int(Freq/2)
    model = EEGNet(input_shape=segment_shape,
               num_class=2,
               loss='sparse_categorical_crossentropy',
               optimizer=Adam(beta_1=0.9, beta_2=0.999, epsilon=1e-08),
               epochs=500,
               batch_size=100,
               es_patience=200,
               monitor='val_accuracy',
               log_path = f'logs/Independant',
               dropout_rate = dropout_rate,
               kernLength = int(Freq/2),
               lr = lr,
               min_lr = lr/10,  
               shuffle = True)
    
    # Fit data to model
    model.fit(PP_data['train_valid'], label_ndarrays_CInonCI['train_valid'], PP_data['test'], label_ndarrays_CInonCI['test'])
    
    # evaluation
    keys = data.keys()
    for key in keys:
        if (key != 'train') | (key != 'valid'):
            Y,evaluation = model.predict(PP_data[key], label_ndarrays_CInonCI[key])
            evaluation_perFold[key].append(evaluation)
            Y_perFold[key].append(Y)
        else: pass
    
    # save results
    for key, value in evaluation_perFold.items():
        evaluation_perFold[key] = pd.DataFrame(value)
    
    folder_path = f'./Results/EEGNet/5Fold test(k_length={k_length}, Fs={target_Freq})/z-score={z_score}, minmax={minmax}, at_each_time_point={at_each_time_point}, across_channel={across_channel}/{dropout_rate},{lr},conduct_Czrrf={conduct_Czrrf}/independant test'
    if not os.path.exists(folder_path):    os.makedirs(folder_path)
    
    # save evaluation to .csv
    [value.to_csv(os.path.join(folder_path, f'evaluation_{key}.csv')) for key, value in evaluation_perFold.items()]
    
    # save true and predicted label to pickle
    with open(os.path.join(folder_path, f'Y_perFold.pkl'), 'wb') as handle:
        pickle.dump(Y_perFold, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # save subject-based evaluation to .csv
    subject_correct = {}
    subject_accuracy = {}
    subject_true_label = {}
    subject_pred_label = {}
    for key, value in Y_perFold.items():
        subject_accuracy[key] = []
        for Y in Y_perFold[key]:
            subject_correct[key], accuracy, subject_true_label[key], subject_pred_label[key] = subject_based_accuracy(Y)
            subject_accuracy[key].append(accuracy)
        subject_accuracy[key] = pd.DataFrame(subject_accuracy[key])
        subject_accuracy[key].to_csv(os.path.join(folder_path, f'subject-based evaluation_{key}.csv'))
        print(f'{key}:{subject_accuracy[key]}')

ModuleNotFoundError: No module named 'EEGNet_function'

### TPR, TNR computation
- load the Y_perfold.pkl data
- compute TPR, TNR for test
- (develping) compute average TPR, TNR among all fold

In [5]:
# load true and predicted label to pickle
folder_path = f'./Results/EEGNet/5Fold test(k_length={k_length}, Fs={Freq})/{dropout_rate},{lr},CZrrf'
print(f'folder_path:{folder_path}')
with open(os.path.join(folder_path, 'Y_perFold.pkl'), 'rb') as handle:
    Y_perFold = pickle.load(handle)

from sklearn.metrics import confusion_matrix

# save subject-based evaluation to .csv
subject_correct = {}
subject_accuracy = {}
subject_true_label = {}
subject_pred_label = {}
for key, value in Y_perFold.items():
    subject_accuracy[key] = []
    for Y in Y_perFold[key]:
        subject_correct[key], accuracy, subject_true_label[key], subject_pred_label[key] = subject_based_accuracy(Y)
        subject_accuracy[key].append(accuracy)
    subject_accuracy[key] = pd.DataFrame(subject_accuracy[key])
    # subject_accuracy[key].to_csv(os.path.join(folder_path, f'subject-based evaluation_{key}.csv'))
    print(f'{key}:{subject_accuracy[key]}')
    
# ------------------------------------------------------------------------------
# test results
print(subject_correct['test'].shape)
subject_true_label = info['test'].Label.values
print(subject_true_label)
subject_true_label = np.where(subject_true_label == 2, 1, subject_true_label)
print(subject_true_label.shape)

subject_predic_label = []
for i, correct in enumerate(subject_correct['test']):
    # print(i,correct)
    if correct == 1:
        subject_predic_label.append(subject_true_label[i])
    else:
        subject_predic_label.append(np.abs(1-subject_true_label[i]))
subject_predic_label = np.array(subject_predic_label)

print(subject_true_label)
print(subject_correct['test'])
print(subject_predic_label)

confusion_matrix = confusion_matrix(subject_true_label, subject_predic_label)
print(confusion_matrix)
TN, FP, FN, TP = confusion_matrix.ravel()
print(f'TN:{TN}, FP:{FP}, FN:{FN}, TP:{TP}')

TPR = TP/(TP+FN)
TNR = TN/(TN+FP)
print('TPR, TNR:',TPR, TNR)

NameError: name 'k_length' is not defined

In [ ]:
# load true and predicted label to pickle
k_length = 62
Freq = 125
folder_path = f'./Results/EEGNet/5Fold test(k_length={k_length}, Fs={Freq})/{dropout_rate},{lr}'
print(f'folder_path:{folder_path}')
with open(os.path.join(folder_path, 'Y_perFold.pkl'), 'rb') as handle:
    Y_perFold = pickle.load(handle)

from sklearn.metrics import confusion_matrix

# save subject-based evaluation to .csv
subject_correct = {}
subject_accuracy = {}
subject_true_label = {}
subject_pred_label = {}
subject_TPR = {}
subject_TNR = {}
for key, value in Y_perFold.items():
    subject_accuracy[key] = []
    subject_TPR[key] = []
    subject_TNR[key] = []
    for Y in Y_perFold[key]:
        subject_correct[key], accuracy, subject_true_label[key], subject_pred_label[key] = subject_based_accuracy(Y)
        # print(f'subject_true_label[{key}]:{type(subject_true_label[key])}\n',subject_true_label[key])
        # print(f'subject_pred_label[{key}]:{type(subject_pred_label[key])}\n',subject_pred_label[key])
        confusion_matrix_object = confusion_matrix(subject_true_label[key], subject_pred_label[key])
        # print(confusion_matrix_object)
        TN, FP, FN, TP = confusion_matrix_object.ravel()
        # print(f'TN:{TN}, FP:{FP}, FN:{FN}, TP:{TP}')
        
        TPR = TP/(TP+FN)
        TNR = TN/(TN+FP)
        # print('TPR, TNR:',TPR, TNR)
        
        subject_accuracy[key].append(accuracy)
        subject_TPR[key].append(TPR)
        subject_TNR[key].append(TNR)

    print('---------------------------------------')
    subject_accuracy[key] = pd.DataFrame(subject_accuracy[key])
    # subject_accuracy[key].to_csv(os.path.join(folder_path, f'subject-based evaluation_{key}.csv'))
    print(f'subject_accuracy of {key}:\n{subject_accuracy[key]}')
    print(f'average:{subject_accuracy[key].mean()}')

    subject_TPR[key] = pd.DataFrame(subject_TPR[key])
    # subject_accuracy[key].to_csv(os.path.join(folder_path, f'subject-based evaluation_{key}.csv'))
    print(f'subject_TPR of {key}:\n{subject_TPR[key]}')
    print(f'average:{subject_TPR[key].mean()}')

    subject_TNR[key] = pd.DataFrame(subject_TNR[key])
    # subject_accuracy[key].to_csv(os.path.join(folder_path, f'subject-based evaluation_{key}.csv'))
    print(f'subject_TNR of {key}:\n{subject_TNR[key]}')
    print(f'average:{subject_TNR[key].mean()}')
